In [2]:
pip install sentence_transformers

     ---------------------------------------- 86.0/86.0 kB 4.7 MB/s eta 0:00:00
  Preparing metadata (setup.py): started
  Preparing metadata (setup.py): finished with status 'done'
     ---------------------------------------- 1.3/1.3 MB 7.3 MB/s eta 0:00:00
     -------------------------------------- 977.6/977.6 kB 6.2 MB/s eta 0:00:00
     -------------------------------------- 192.2/192.2 MB 2.3 MB/s eta 0:00:00
  Created wheel for sentence_transformers: filename=sentence_transformers-2.2.2-py3-none-any.whl size=125925 sha256=d9dbe74d7ae2eb262f2954c8f1de7e25d5a0183a693c5c90fb622cb8712ee765
  Stored in directory: c:\users\yfr\appdata\local\pip\cache\wheels\71\67\06\162a3760c40d74dd40bc855d527008d26341c2b0ecf3e8e11f
Successfully built sentence_transformers
  Attempting uninstall: torch
    Found existing installation: torch 2.0.1
    Uninstalling torch-2.0.1:
      Successfully uninstalled torch-2.0.1
Note: you may need to restart the kernel to use updated packages.


In [6]:
from sentence_transformers import SentenceTransformer, util
import os
import shutil
import pandas as pd
import numpy as np
import torch
import torch.nn.functional as F
from nltk import tokenize

In [7]:
model = SentenceTransformer('all-mpnet-base-v2')

In [8]:
# enter the query here
q = 'Can you give me some tenders related to cleaning services'
query_embedding = model.encode(q)

In [9]:
# get sentence transformer embedding
em = np.load('sent_transformer_embeddings.npz')
sBert_em = [em[f] for f in em.files]
# convert into data frame
tender_df = pd.DataFrame(sBert_em)

In [27]:
import pandas as pd
tenders_structured_path = r"./data/UpdatedAgainTenders.xlsx"

tenders_structured = pd.read_excel(tenders_structured_path)
tenders_structured = tenders_structured[["Reference Number", "Contract Title"]].dropna(subset=["Reference Number"])

In [29]:
tender_refs = open("./data/tender_references.txt", "r")
content = ""
for line in tender_refs:
    content = line
    break
    
content = content.replace("[", "")
content = content.replace("'", "")
content = content.replace(",", "")
refs = content.split(" ")

20619
20619


In [35]:
tender_data = pd.DataFrame(tender_df)
tender_data['Reference Number'] = refs
merged_data = tender_data.merge(tenders_structured, on="Reference Number", how="left")

In [46]:
from sklearn.metrics.pairwise import cosine_similarity

similarities = cosine_similarity(query_embedding.reshape(1, -1), merged_data.iloc[:, :768])

top_10_indices = similarities[0].argsort()[-10:][::-1]

top_10_tenders = merged_data.iloc[top_10_indices]

print("Top 10 most similar tenders:")
for index, row in top_10_tenders.iterrows():
    title = merged_data.iloc[index, -1]  
    similarity_score = similarities[0][index]
    print(f"Tender Title: {title}, Similarity Score: {similarity_score}")

Top 10 most similar tenders:
Tender Title: Provision of Cleaning Services, Similarity Score: 0.7198266983032227
Tender Title: Cleaning Services - Northam Regional Office, Similarity Score: 0.7125318646430969
Tender Title: Provision of Cleaning Services for the Department of Primary industries and Regional Development Karratha District Office, Similarity Score: 0.7109395265579224
Tender Title: Provision of Cleaning Services ? 72 Walters Drive, Osborne Park, 6017, Similarity Score: 0.7089492678642273
Tender Title: Provision of Cleaning Services, Similarity Score: 0.6979150772094727
Tender Title: Request for Cleaning Services to Clean Department of Transport Broome Office, Similarity Score: 0.6953589916229248
Tender Title: Provision of Cleaning Services for South Regional TAFE, Esperance Campus, Similarity Score: 0.6941353678703308
Tender Title: Cleaning Services for Karratha Office, Similarity Score: 0.6936020851135254
Tender Title: Provision of Cleaning Services for the Department of Pr

In [11]:
def find_top_similar_tenders(query, model, merged_data, top_k=10):
    # Encode the query
    query_embedding = model.encode(query)
    
    # Calculate cosine similarities
    similarities = cosine_similarity(query_embedding.reshape(1, -1), merged_data.iloc[:, :768])
    
    # Get the indices of the top k similar tenders
    top_k_indices = similarities[0].argsort()[-top_k:][::-1]
    
    # Create a DataFrame to store the results
    result_df = pd.DataFrame(columns=["Tender Title", "Similarity Score"])
    
    for index in top_k_indices:
        title = merged_data.iloc[index, -1]
        similarity_score = similarities[0][index]
        result_df = result_df.append({"Tender Title": title, "Similarity Score": similarity_score}, ignore_index=True)
    
    return result_df